# Import Dataset

In [1]:
import pandas as pd
import re
import requests

# https://www.kaggle.com/datasets/promptcloud/amazon-product-dataset-2020/data
dataset = pd.read_csv(r"marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv")
dataset.head(5)

,Uniq Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,Model Number,...,Product Url,Stock,Product Details,Dimensions,Color,Ingredients,Direction To Use,Is Amazon Seller,Size Quantity Variant,Product Description
0,4c69b61db1fc16e7013b43fc926e502d,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",NaN,NaN,Sports & Outdoors | Outdoor Recreation | Skate...,NaN,NaN,$237.68,NaN,NaN,...,https://www.amazon.com/DB-Longboards-CoreFlex-...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
1,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",NaN,NaN,Toys & Games | Learning & Education | Science ...,NaN,NaN,$99.95,NaN,55324,...,https://www.amazon.com/Electronic-Circuits-Cla...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
2,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,NaN,NaN,Toys & Games | Arts & Crafts | Craft Kits,NaN,NaN,$34.99,NaN,NaN,...,https://www.amazon.com/3Doodler-Plastic-Innova...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
3,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,NaN,NaN,Toys & Games | Hobbies | Models & Model Kits |...,NaN,NaN,$28.91,NaN,142,...,https://www.amazon.com/Guillow-Airplane-Design...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
4,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,NaN,NaN,Toys & Games | Puzzles | Jigsaw Puzzles,NaN,NaN,$17.49,NaN,62151,...,https://www.amazon.com/Woodstock-Collage-500-p...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN


# Create, Save, and Load Priority Products to Collect GTINs For

In [5]:
# Function to extract brand from URL
def extract_brand(url):
    match = re.search(r'amazon\.com/([A-Z]+)(?![a-z])', url)
    if match:
        return match.group(1)
    else:
        return None


# Apply function to create new column
dataset['Brand'] = dataset['Product Url'].apply(extract_brand)
dataset[['Product Url','Brand']]

priority = dataset[
    (dataset['Brand'].isna()==False)&
    (dataset['Model Number'].isna()==False)&
    (dataset['Upc Ean Code'].isna()==True)
    ][['Brand','Model Number','Product Name']]

priority.columns = [["brand","mpn","title"]]
priority.to_csv('priority.csv')
priority.head()

,brand,mpn,title
8,ARTSCAPE,01-0121,"ARTSCAPE Etched Glass 24"" x 36"" Window Film, 2..."
11,DC,JAN190710,DC Cover Girls: Black Canary by Joëlle Jones S...
13,LEGO,6288704,LEGO Minecraft Creeper BigFig and Ocelot Chara...
22,PLUS,03373,"PLUS PLUS - Construction Building Toy, Open Pl..."
26,NKOK,RC-611,NKOK Sonic and Sega All Stars Racing Remote Co...


# Load Logs Already Requested

In [10]:
log = pd.read_csv("logs.csv", index_col=False)
log.head()

,brand,mpn,title,api_response,gtin,gtin_exists
0,NaN,NaN,NaN,NaN,NaN,NaN


# Create List of Products not yet run through Produkter

In [11]:
merged_df = pd.merge(priority, log, left_on='mpn', right_on='mpn', how='left', indicator=True, suffixes=('_priority', '_log'))
merged_df
# # Filter rows that exist in the first dataframe but not in the second
# result_df = merged_df[merged_df['_merge'] == 'left_only']

# # Drop the '_merge' column as it's no longer needed
# result_df = result_df.drop(columns=['_merge'])

# # Drop duplicate '_y' columns
# result_df = result_df.loc[:, ~result_df.columns.str.endswith('_y')]

# # Rename remaining columns to remove '_x' suffixes
# todo = result_df.rename(columns=lambda x: x[:-2] if x.endswith('_x') else x)
# todo

ValueError: The column label 'mpn' is not unique.
For a multi-index, the label must be a tuple with elements corresponding to each level.

# Format Functions

In [ ]:
def get_gtin_values(response_json, q, gtin_key='gtin'):
    try:
        gtin_values = response_json.get(q, {}).get(gtin_key, [])
        
        # Filter out None values and strings with length < 5 characters
        filtered_values = [value for value in gtin_values if value is not None and (not isinstance(value, str) or len(value) > 5)]
    
    except: # no response_json
        filtered_values = []    

    return filtered_values
# get_gtin_values(data, q, gtin_key='gtin12')


def filter_strings(*lists):
    result = []
    for lst in lists:
        for item in lst:
            if isinstance(item, str) and len(item) > 5:
                result.append(item)
    return result


def any_list_non_empty(*lists):
    for l in lists:
        if l:
            return True
    return False

# Collect GTINs

In [ ]:
url = "https://produkter.p.rapidapi.com/schemas"

headers = {
	"X-RapidAPI-Key": "XXXXXXXXXXXXX",
	"X-RapidAPI-Host": "produkter.p.rapidapi.com"
}

for i, row in df.iloc[:10,:].iterrows():
	querystring = {"q":f"{row['Brand']} {row['Model Number']}"}		
	response = requests.get(url, headers=headers, params=querystring)
	print(response.json())

In [ ]:
# Initialize `log` column variables for each `todo` row
i_brand = None
i_mpn = None
i_title = None
i_api_response = None
i_gtin = None
i_gtin8 = None
i_gtin12 = None
i_gtin13 = None
i_gtin_exists = None

# Initialize a list to store rows
output_rows = []

for i, row in todo.iloc[:50, :].iterrows():
    
    i_brand = row['brand']
    i_mpn = row['mpn']
    i_title = row['title']
    print(i_brand,i_mpn)

    querystring = {"q": f"{i_brand} {i_mpn}"}

    # try requests + store i_api_response
    try:
        # Set a timeout of 25 seconds for the request
        response = requests.get(url, headers=headers, params=querystring, timeout=35)
        try:
            response_json = response.json()
            # Check response status code
            if response.status_code == 200:
                i_api_response = "200"
            else:
                i_api_response = "not 200"
        except Exception as e:
            response_json = None
            i_api_response = e
            
    except requests.Timeout:
        # Request timed out
        response_json = None
        i_api_response = "timeout"

        

    # get gtins
    try:
        i_gtin = get_gtin_values(response_json, q=f"{i_brand} {i_mpn}", gtin_key='gtin')
        i_gtin8 = get_gtin_values(response_json, q=f"{i_brand} {i_mpn}", gtin_key='gtin8')
        i_gtin12 = get_gtin_values(response_json, q=f"{i_brand} {i_mpn}", gtin_key='gtin12')
        i_gtin13 = get_gtin_values(response_json, q=f"{i_brand} {i_mpn}", gtin_key='gtin13')
        
        filter_strings(i_gtin,i_gtin8,i_gtin12,i_gtin13)

        i_gtin_exists = any_list_non_empty(i_gtin,i_gtin8,i_gtin12,i_gtin13)

    except Exception as e:
        # Handle any errors here
        print(f"Error processing gtins on row {i} ({i_brand} {i_mpn}): \n {e}")
        i_gtin = []
        i_gtin8 = []
        i_gtin12 = []
        i_gtin13 = []
        
        i_gtin_exists = any_list_non_empty(i_gtin,i_gtin8,i_gtin12,i_gtin13)

    
    # make new rows
    new_row = {
        'zoroNo':i_zoroNo,
        'brand':i_brand,
        'mpn':i_mpn,
        'title':i_title,
        'api_response':i_api_response,
        'gtin':i_gtin,
        'gtin8':i_gtin8,
        'gtin12':i_gtin12,
        'gtin13':i_gtin13,
        'gtin_exists':i_gtin_exists
    }

    # Append the dictionary to the list
    output_rows.append(new_row)

    time.sleep(5)

In [ ]:
new_logs = pd.DataFrame(output_rows)
new_logs

In [ ]:
# Create a new DataFrame from the list of dictionaries
new_logs = pd.DataFrame(output_rows)

new_logs.to_csv('./requested_logs.csv', index=False, mode='a', header=False)

In [ ]:
# Cleanup duplicates if they exist

# Read the updated CSV file into a DataFrame
log = pd.read_csv('./requested_logs.csv')

# Drop duplicates in the "zoroNo" column, keeping the record where "gtin_exists" is True if duplicates exist
log = log.sort_values('gtin_exists', ascending=False).drop_duplicates(subset='zoroNo')

# Write the filtered DataFrame back to the CSV file
log.to_csv('./requested_logs.csv', index=False)

# Report on GTIN Collection

import plotly.express as px
log = pd.read_csv("logs.csv", index_col=False)
log

In [ ]:
api_response = log.groupby(by="api_response")['mpn'].count()
api_response = api_response.rename('count').reset_index().rename(columns={'api_response': 'api_response'})

print(f"SKUs Requested: {sum(api_response['count'])}")
display(api_response)

fig = px.bar(api_response, x="api_response", y='count',title="GTIN Collection Logs | API Responses")

# Annotate each bar with the percentage
for i in range(len(api_response)):
    total_count = api_response['count'].sum()
    percentage = (api_response.iloc[i]['count'] / total_count) * 100
    fig.add_annotation(x=api_response.index[i], y=api_response.iloc[i]['count'] + 10, 
                       text=f"{percentage:.2f}%", showarrow=False)

fig.show()

In [ ]:
gtin_exists = log.groupby(by="gtin_exists")['mpn'].count()
gtin_exists = gtin_exists.rename('count').reset_index().rename(columns={'gtin_exists': 'gtin_exists'})

print(f"SKUs Requested: {sum(gtin_exists['count'])}")
display(gtin_exists)

fig = px.bar(gtin_exists, x="gtin_exists", y='count',title="GTIN Collection Logs | GTINs Found")

# Annotate each bar with the percentage
for i in range(len(gtin_exists)):
    total_count = gtin_exists['count'].sum()
    percentage = (gtin_exists.iloc[i]['count'] / total_count) * 100
    fig.add_annotation(x=gtin_exists.index[i], y=gtin_exists.iloc[i]['count'] + 10, 
                       text=f"{percentage:.2f}%", showarrow=False)

fig.show()